In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import math

# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Apply label encoding to the 'state' column
label_encoder = LabelEncoder()
df['state'] = label_encoder.fit_transform(df['state'])

df.tail()

,client_id,sex,age,state,risk_profile,invest_goals,temp_goals,fin_exp,fin_avail,a1,...,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20
1166,5146755,1,69,25,4,14,17,10,19,0.167,...,0.0,0.095,0.0,0.020,0.0,0.0,0.181,0.000,0.000,0.0
1167,5738347,1,56,11,3,13,17,11,19,0.034,...,0.0,0.065,0.0,0.068,0.0,0.0,0.000,0.065,0.049,0.0
1168,8516725,1,64,24,4,13,17,11,19,0.178,...,0.0,0.000,0.0,0.055,0.0,0.0,0.023,0.000,0.000,0.0
1169,14240215,1,57,11,3,13,17,10,19,0.009,...,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.000,0.0
1170,14326270,0,51,14,3,13,17,11,19,0.184,...,0.0,0.000,0.0,0.006,0.0,0.0,0.018,0.000,0.000,0.0


In [2]:
# 'data' contains columns 'sex', 'age', 'state', 'risk_profile'
data = df[['sex', 'age', 'state', 'risk_profile']].values

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# 'targets' contains columns 'a1' through 'a20'
targets = df[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20']].values

# Split the data into training and testing sets
split_ratio = 0.7  # 70% for training, 30% for testing
split_index = int(len(data) * split_ratio)

X_train, X_test = data[:split_index], data[split_index:]
y_train, y_test = targets[:split_index], targets[split_index:]

print('X_train shape =', X_train.shape)
print('y_train shape =', y_train.shape)
print('X_test shape =', X_test.shape)
print('y_test shape =', y_test.shape)

X_train shape = (819, 4)
y_train shape = (819, 20)
X_test shape = (352, 4)
y_test shape = (352, 20)


In [3]:
# Create a decision tree regressor
### model = DecisionTreeRegressor()
model = DecisionTreeRegressor(max_depth=3, min_samples_split=2, min_samples_leaf=1)

# Train the model on the training data
model.fit(X_train, y_train)

# Display the model parameters
print("Final Model Parameters:")
print(f"Max Depth: {model.max_depth}")
print(f"Min Samples Split: {model.min_samples_split}")
print(f"Min Samples Leaf: {model.min_samples_leaf}")

Final Model Parameters:
Max Depth: 3
Min Samples Split: 2
Min Samples Leaf: 1


In [4]:
# Make predictions on the testing data
y_pred = model.predict(X_test)

print('y_test shape =', y_test.shape)
print('y_pred shape =', y_pred.shape)

y_test shape = (352, 20)
y_pred shape = (352, 20)


In [5]:
sums = []
for row in y_test:
    sums.append(sum(row))
#print('y_test =', sums)
print("y_test average =", sum(sums)/len(sums))

y_test average = 1.0000227272727267


In [6]:
sums = []
for row in y_pred:
    sums.append(sum(row))
#print('y_pred =', sums)
print("y_pred average =", sum(sums)/len(sums))

y_pred average = 1.0000414780489857


In [7]:
# Remove values < 0.01 and Scale predictions to get 1 (100%) as a sum of each row
num_rows = len(y_pred)
num_cols = len(y_pred[0])

for row in range(num_rows):
    for col in range(num_cols):
        if y_pred[row][col] < 0.01:
            y_pred[row][col] = 0

for row in range(num_rows):
    row_sum = sum(y_pred[row])
    for col in range(num_cols):
        y_pred[row][col] = y_pred[row][col] / row_sum

In [8]:
sums = []
for row in y_pred:
    sums.append(sum(row))
#print('y_pred =', sums)
print("y_pred average =", sum(sums)/len(sums))

y_pred average = 1.0


In [9]:
data_for_csv = df[['sex', 'age', 'state', 'risk_profile']].values
data_train, data_test = data_for_csv[:split_index], data_for_csv[split_index:]
array1 = data_test
array2 = np.round(np.array(y_pred), 3)

result_array = np.concatenate((array1, array2), axis=1)

np.set_printoptions(suppress=True)
print(result_array)

np.savetxt('RT_output.csv', result_array, delimiter=",", fmt='%.3f')

[[ 1.    41.    33.    ...  0.     0.     0.   ]
 [ 1.    69.    14.    ...  0.     0.     0.   ]
 [ 1.    46.    28.    ...  0.     0.     0.   ]
 ...
 [ 1.    64.    24.    ...  0.     0.     0.   ]
 [ 1.    57.    11.    ...  0.     0.     0.   ]
 [ 0.    51.    14.    ...  0.012  0.     0.   ]]
